# How to track token usage in ChatModels

:::info Prerequisites

This guide assumes familiarity with the following concepts:
- [Chat models](/docs/concepts/#chat-models)

:::

Tracking token usage to calculate cost is an important part of putting your app in production. This guide goes over how to obtain this information from your LangChain model calls.

This guide requires `langchain-openai >= 0.1.9`.

In [1]:
%pip install --upgrade --quiet langchain langchain-openai

Note: you may need to restart the kernel to use updated packages.


## Using LangSmith

You can use [LangSmith](https://www.langchain.com/langsmith) to help track token usage in your LLM application. See the [LangSmith quick start guide](https://docs.smith.langchain.com/).

## Using AIMessage.usage_metadata

A number of model providers return token usage information as part of the chat generation response. When available, this information will be included on the `AIMessage` objects produced by the corresponding model.

LangChain `AIMessage` objects include a [usage_metadata](https://python.langchain.com/v0.2/api_reference/core/messages/langchain_core.messages.ai.AIMessage.html#langchain_core.messages.ai.AIMessage.usage_metadata) attribute. When populated, this attribute will be a [UsageMetadata](https://python.langchain.com/v0.2/api_reference/core/messages/langchain_core.messages.ai.UsageMetadata.html) dictionary with standard keys (e.g., `"input_tokens"` and `"output_tokens"`).

Examples:

**OpenAI**:

In [2]:
# # !pip install -qU langchain-openai

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
openai_response = llm.invoke("hello")
openai_response.usage_metadata

{'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17}

**Anthropic**:

In [3]:
# !pip install -qU langchain-anthropic

from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="claude-3-haiku-20240307")
anthropic_response = llm.invoke("hello")
anthropic_response.usage_metadata

{'input_tokens': 8, 'output_tokens': 12, 'total_tokens': 20}

### Using AIMessage.response_metadata

Metadata from the model response is also included in the AIMessage [response_metadata](https://python.langchain.com/v0.2/api_reference/core/messages/langchain_core.messages.ai.AIMessage.html#langchain_core.messages.ai.AIMessage.response_metadata) attribute. These data are typically not standardized. Note that different providers adopt different conventions for representing token counts:

In [4]:
print(f'OpenAI: {openai_response.response_metadata["token_usage"]}\n')
print(f'Anthropic: {anthropic_response.response_metadata["usage"]}')

OpenAI: {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17}

Anthropic: {'input_tokens': 8, 'output_tokens': 12}


### Streaming

Some providers support token count metadata in a streaming context.

#### OpenAI

For example, OpenAI will return a message [chunk](https://python.langchain.com/v0.2/api_reference/core/messages/langchain_core.messages.ai.AIMessageChunk.html) at the end of a stream with token usage information. This behavior is supported by `langchain-openai >= 0.1.9` and can be enabled by setting `stream_usage=True`. This attribute can also be set when `ChatOpenAI` is instantiated.

```{=mdx}
:::note
By default, the last message chunk in a stream will include a `"finish_reason"` in the message's `response_metadata` attribute. If we include token usage in streaming mode, an additional chunk containing usage metadata will be added to the end of the stream, such that `"finish_reason"` appears on the second to last message chunk.
:::
```

In [5]:
llm = ChatOpenAI(model="gpt-4o-mini")

aggregate = None
for chunk in llm.stream("hello", stream_usage=True):
    print(chunk)
    aggregate = chunk if aggregate is None else aggregate + chunk

content='' additional_kwargs={} response_metadata={} id='run-c8896c9b-d62b-4e02-bfda-557eefd52d5a'
content='Hello' additional_kwargs={} response_metadata={} id='run-c8896c9b-d62b-4e02-bfda-557eefd52d5a'
content='!' additional_kwargs={} response_metadata={} id='run-c8896c9b-d62b-4e02-bfda-557eefd52d5a'
content=' How' additional_kwargs={} response_metadata={} id='run-c8896c9b-d62b-4e02-bfda-557eefd52d5a'
content=' can' additional_kwargs={} response_metadata={} id='run-c8896c9b-d62b-4e02-bfda-557eefd52d5a'
content=' I' additional_kwargs={} response_metadata={} id='run-c8896c9b-d62b-4e02-bfda-557eefd52d5a'
content=' assist' additional_kwargs={} response_metadata={} id='run-c8896c9b-d62b-4e02-bfda-557eefd52d5a'
content=' you' additional_kwargs={} response_metadata={} id='run-c8896c9b-d62b-4e02-bfda-557eefd52d5a'
content=' today' additional_kwargs={} response_metadata={} id='run-c8896c9b-d62b-4e02-bfda-557eefd52d5a'
content='?' additional_kwargs={} response_metadata={} id='run-c8896c9b-d62b-

Note that the usage metadata will be included in the sum of the individual message chunks:

In [6]:
print(aggregate.content)
print(aggregate.usage_metadata)

Hello! How can I assist you today?
{'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17}


To disable streaming token counts for OpenAI, set `stream_usage` to False, or omit it from the parameters:

In [7]:
aggregate = None
for chunk in llm.stream("hello"):
    print(chunk)

content='' additional_kwargs={} response_metadata={} id='run-1d3bd183-2557-40c6-b24d-c4b4caf9749e'
content='Hello' additional_kwargs={} response_metadata={} id='run-1d3bd183-2557-40c6-b24d-c4b4caf9749e'
content='!' additional_kwargs={} response_metadata={} id='run-1d3bd183-2557-40c6-b24d-c4b4caf9749e'
content=' How' additional_kwargs={} response_metadata={} id='run-1d3bd183-2557-40c6-b24d-c4b4caf9749e'
content=' can' additional_kwargs={} response_metadata={} id='run-1d3bd183-2557-40c6-b24d-c4b4caf9749e'
content=' I' additional_kwargs={} response_metadata={} id='run-1d3bd183-2557-40c6-b24d-c4b4caf9749e'
content=' assist' additional_kwargs={} response_metadata={} id='run-1d3bd183-2557-40c6-b24d-c4b4caf9749e'
content=' you' additional_kwargs={} response_metadata={} id='run-1d3bd183-2557-40c6-b24d-c4b4caf9749e'
content=' today' additional_kwargs={} response_metadata={} id='run-1d3bd183-2557-40c6-b24d-c4b4caf9749e'
content='?' additional_kwargs={} response_metadata={} id='run-1d3bd183-2557-

You can also enable streaming token usage by setting `stream_usage` when instantiating the chat model. This can be useful when incorporating chat models into LangChain [chains](/docs/concepts#langchain-expression-language-lcel): usage metadata can be monitored when [streaming intermediate steps](/docs/how_to/streaming#using-stream-events) or using tracing software such as [LangSmith](https://docs.smith.langchain.com/).

See the below example, where we return output structured to a desired schema, but can still observe token usage streamed from intermediate steps.

In [8]:
from pydantic import BaseModel, Field


class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


llm = ChatOpenAI(
    model="gpt-4o-mini",
    stream_usage=True,
)
# Under the hood, .with_structured_output binds tools to the
# chat model and appends a parser.
structured_llm = llm.with_structured_output(Joke)

async for event in structured_llm.astream_events("Tell me a joke", version="v2"):
    if event["event"] == "on_chat_model_end":
        print(f'Token usage: {event["data"]["output"].usage_metadata}\n')
    elif event["event"] == "on_chain_end":
        print(event["data"]["output"])
    else:
        pass

Token usage: {'input_tokens': 74, 'output_tokens': 21, 'total_tokens': 95}

setup="Why don't scientists trust atoms?" punchline='Because they make up everything!'


Token usage is also visible in the corresponding [LangSmith trace](https://smith.langchain.com/public/fe6513d5-7212-4045-82e0-fefa28bc7656/r) in the payload from the chat model.

## Using callbacks

There are also some API-specific callback context managers that allow you to track token usage across multiple calls. It is currently only implemented for the OpenAI API and Bedrock Anthropic API.

### OpenAI

Let's first look at an extremely simple example of tracking token usage for a single Chat model call.

In [9]:
# !pip install -qU langchain-community wikipedia

from langchain_community.callbacks.manager import get_openai_callback

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    stream_usage=True,
)

with get_openai_callback() as cb:
    result = llm.invoke("Tell me a joke")
    print(cb)

Tokens Used: 29
	Prompt Tokens: 11
	Completion Tokens: 18
Successful Requests: 1
Total Cost (USD): $1.2449999999999998e-05


Anything inside the context manager will get tracked. Here's an example of using it to track multiple calls in sequence.

In [10]:
with get_openai_callback() as cb:
    result = llm.invoke("Tell me a joke")
    result2 = llm.invoke("Tell me a joke")
    print(cb.total_tokens)

58


In [11]:
with get_openai_callback() as cb:
    for chunk in llm.stream("Tell me a joke"):
        pass
    print(cb)

Tokens Used: 29
	Prompt Tokens: 11
	Completion Tokens: 18
Successful Requests: 1
Total Cost (USD): $1.2449999999999998e-05


If a chain or agent with multiple steps in it is used, it will track all those steps.

In [12]:
from langchain.agents import AgentExecutor, create_tool_calling_agent, load_tools
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You're a helpful assistant"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)
tools = load_tools(["wikipedia"])
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [13]:
with get_openai_callback() as cb:
    response = agent_executor.invoke(
        {
            "input": "What's a hummingbird's scientific name and what's the fastest bird species?"
        }
    )
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")



> Entering new AgentExecutor chain...



Invoking: `wikipedia` with `{'query': 'Hummingbird'}`




Page: Hummingbird
Summary: Hummingbirds are birds native to the Americas and comprise the biological family Trochilidae. With approximately 366 species and 113 genera, they occur from Alaska to Tierra del Fuego, but most species are found in Central and South America. As of 2024, 21 hummingbird species are listed as endangered or critically endangered, with numerous species declining in population.
Hummingbirds have varied specialized characteristics to enable rapid, maneuverable flight: exceptional metabolic capacity, adaptations to high altitude, sensitive visual and communication abilities, and long-distance migration in some species. Among all birds, male hummingbirds have the widest diversity of plumage color, particularly in blues, greens, and purples. Hummingbirds are the smallest mature birds, measuring 7.5–13 cm (3–5 in) in length. The smallest is the 5 cm (2.0 in) bee hummingbird, which weighs less than 2.0 g (0.07 oz), and the largest is the 23 cm (9 in) giant hummingbird, w

Page: List of birds by flight speed
Summary: This is a list of the fastest flying birds in the world. A bird's velocity is necessarily variable; a hunting bird will reach much greater speeds while diving to catch prey than when flying horizontally. The bird that can achieve the greatest airspeed is the peregrine falcon (Falco peregrinus), able to exceed 320 km/h (200 mph) in its dives. A close relative of the common swift, the white-throated needletail (Hirundapus caudacutus), is commonly reported as the fastest bird in level flight with a reported top speed of 169 km/h (105 mph). This record remains unconfirmed as the measurement methods have never been published or verified. The record for the fastest confirmed level flight by a bird is 111.5 km/h (69.3 mph) held by the common swift.

Page: Fastest animals
Summary: This is a list of the fastest animals in the world, by types of animal.

Page: Falcon
Summary: Falcons () are birds of prey in the genus Falco, which includes about 40 spe

The scientific name for hummingbirds is **Trochilidae**, which is the biological family that encompasses approximately 366 species of these birds.

The fastest bird species is the **peregrine falcon** (*Falco peregrinus*), which can exceed speeds of 320 km/h (200 mph) during its hunting dives. In terms of level flight, the common swift (*Apus apus*) holds the record for the fastest confirmed speed at 111.5 km/h (69.3 mph).

> Finished chain.
Total Tokens: 1797
Prompt Tokens: 1648
Completion Tokens: 149
Total Cost (USD): $0.00033659999999999994


### Bedrock Anthropic

The `get_bedrock_anthropic_callback` works very similarly:

In [14]:
# !pip install langchain-aws
from langchain_aws import ChatBedrock
from langchain_community.callbacks.manager import get_bedrock_anthropic_callback

llm = ChatBedrock(model_id="anthropic.claude-v2")

with get_bedrock_anthropic_callback() as cb:
    result = llm.invoke("Tell me a joke")
    result2 = llm.invoke("Tell me a joke")
    print(cb)

/Users/bagatur/langchain-aws/libs/aws/langchain_aws/chat_models/__init__.py:1: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_aws.chat_models.bedrock import BedrockChat, ChatBedrock
/Users/bagatur/langchain/.venv/lib/python3.11/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
  warnings.warn(message, UserWarning)


SchemaError: Invalid Schema:
model.config.extra_fields_behavior
  Input should be 'allow', 'forbid' or 'ignore' [type=literal_error, input_value=<Extra.forbid: 'forbid'>, input_type=Extra]
    For further information visit https://errors.pydantic.dev/2.9/v/literal_error

## Next steps

You've now seen a few examples of how to track token usage for supported providers.

Next, check out the other how-to guides chat models in this section, like [how to get a model to return structured output](/docs/how_to/structured_output) or [how to add caching to your chat models](/docs/how_to/chat_model_caching).